In [7]:
using WTP
using SCDM
using Test
using LinearAlgebra

┌ Info: Precompiling WTP [7f2d596f-d764-40ee-883d-fa5ff5538bc6]
└ @ Base loading.jl:1342
┌ Info: Precompiling SCDM [1a6f3412-296b-4a74-a079-535a7d500bdf]
└ @ Base loading.jl:1342


Read the wave functions from the `wfc?.dat` files.

In [8]:
const test_1_dir = "../test/test_data/test_1"
wave_functions_list = wave_functions_from_directory(joinpath(test_1_dir, "si.save"));

In [9]:
u = wannier_from_save(wave_functions_list)
k_map, brillouin_zone = i_kpoint_map(wave_functions_list);
# wave_functions_list = nothing # release the memory

Progress: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Time: 0:00:03


## Notations

### Grid Indexing

Indexing a grid yields a grid vector. 
For example, indexing the Brillouin zone yields a kpoint.

In [11]:
Γ = brillouin_zone[0, 0, 0]

grid: BrillouinZone
_coefficients: [0, 0, 0]


### Band Indexing

Indexing the object `u` with a kpoint and a band index gives an orbital.

In [12]:
u_1_Γ = u[Γ][1]

ket
grid:
    type: ReciprocalLattice
    domain: ((-12, 11), (-12, 11), (-12, 11))
    basis:
        ket: -0.612, -0.612, 0.612
        ket: 0.612, 0.612, 0.612
        ket: -0.612, 0.612, -0.612
    
kpoint:
    grid: BrillouinZone
    _coefficients: [0, 0, 0]
    
band:
    1

### Orbital Indexing

An orbital can be indexed by a grid vector, which can be obtained by indexing a grid.

In [13]:
reciprocal_lattice = grid(u_1_Γ)

type: ReciprocalLattice
domain: ((-12, 11), (-12, 11), (-12, 11))
basis:
    ket: -0.612, -0.612, 0.612
    ket: 0.612, 0.612, 0.612
    ket: -0.612, 0.612, -0.612


In [14]:
orbital_norm = 0
for G in reciprocal_lattice
    orbital_norm += abs2(u_1_Γ[G])
end
orbital_norm

0.99999976f0

### Perform a Fourier transform.

In [15]:
u_real = ifft(u);

### Perform an SCDM.

In [16]:
U = scdm_condense_phase(u_real, collect(1:20));

Progress: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Time: 0:00:01


Verify the result by comparing it to the output of Anil's Matlab code.
We read the `amn` file produced by the Matlab code and contruct a gauge.

In [17]:
amn = AMN(joinpath(test_1_dir, "unk", "si.amn"))
U_matlab = Gauge(brillouin_zone, amn, k_map, false);

In [19]:
for k in brillouin_zone
    @test norm(U[k] - U_matlab[k]) < 1e-5
end

### Centers and Spreads

Compute the centers and the spreads of the Wannier function.
The numeric values are outputs from Wannier90.

Constructing the MMN matrix is in fact a $O(N^2 N_g N_b)$ time algorithm. $N_b$ is the number of neighbors, which can be treated as a constant.

In [20]:
scheme = W90FiniteDifference(u);

Progress: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Time: 0:00:03


In [21]:
M = gauge_transform(neighbor_basis_integral(scheme), U);

In [22]:
@test isapprox(center(M, scheme, 1), [1.751302, -3.656156, -3.154520], atol = 1e-6)
@test isapprox(center(M, scheme, 2), [0.662933, 1.224859, 0.588340], atol = 1e-6)
@test isapprox(center(M, scheme, 3), [0.751350, -1.252297, 0.334295], atol = 1e-6)
@test isapprox(center(M, scheme, 4), [0.745342, 0.390014, -1.239850], atol = 1e-6)

Test Passed

In [23]:
spread(n) = second_moment(M, scheme, n) - norm(center(M, scheme, n))^2

@test isapprox(spread(1), 15.47179726, atol = 1e-6)
@test isapprox(spread(2), 13.17038702, atol = 1e-6)
@test isapprox(spread(3), 7.64407335, atol = 1e-6)
@test isapprox(spread(4), 8.25678268, atol = 1e-6)

Test Passed

In [ ]:
using CairoMakie

In [ ]:
scatter(1:20, spread.(1:20))